This notebook can be used to run an example image through a series of pertubations as defined by the parameters provided to the pybsm package. In order to use this notebook please install all the dependencies as described in the README

**PyBSM Generator will be added once it is integrated the full pertubation pipeline functionality will be added**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pybsm
import copy
from nrtk.impls.perturb_pybsm.StoreSensor import StoreSensor
from nrtk.impls.perturb_pybsm.StoreScenario import StoreScenario

First we define the StoreSensor parameters. These parameters describe the sensor that was used to capture the original image.

In [2]:
#dirpath = os.path.dirname(os.path.abspath(__file__))
imgfile = './data/M-41 Walker Bulldog (USA) width 319cm height 272cm.tiff'

name = 'L32511x'

#telescope focal length (m)
f=4                     
# Telescope diameter (m)
D=275e-3               

#detector pitch (m)
p=.008e-3    
       
#Optical system transmission, red  band first (m)
optTransWavelengths = np.array([0.58-.08,0.58+.08])*1.0e-6
#guess at the full system optical transmission (excluding obscuration)
opticsTransmission=0.5*np.ones(optTransWavelengths.shape[0]) 

# Relative linear telescope obscuration
eta=0.4 #guess     

#detector width is assumed to be equal to the pitch
wx=p                    
wy=p 
#integration time (s) - this is a maximum, the actual integration time will be
#determined by the well fill percentage
intTime=30.0e-3 

#dark current density of 1 nA/cm2 guess, guess mid range for a silicon camera
darkCurrent = pybsm.darkCurrentFromDensity(1e-5,wx,wy)

#rms read noise (rms electrons)
readNoise=25.0 

#maximum ADC level (electrons)
maxN=96000.0

#bit depth
bitdepth=11.9

#maximum allowable well fill (see the paper for the logic behind this)
maxWellFill = .6

#jitter (radians) - The Olson paper says that its "good" so we'll guess 1/4 ifov rms
sx = 0.25*p/f
sy = sx

#drift (radians/s) - again, we'll guess that it's really good
dax = 100e-6
day = dax

#etector quantum efficiency as a function of wavelength (microns) 
#for a generic high quality back-illuminated silicon array
# https://www.photometrics.com/resources/learningzone/quantumefficiency.php
qewavelengths=np.array([.3, .4, .5, .6, .7, .8, .9, 1.0, 1.1])*1.0e-6
qe=np.array([0.05, 0.6, 0.75, 0.85, .85, .75, .5, .2, 0])

sensor = StoreSensor(name, D, f, p, optTransWavelengths,
                                opticsTransmission, eta, wx, wy,
                                intTime, darkCurrent, readNoise,
                                maxN, bitdepth, maxWellFill, sx, sy,
                                dax, day, qewavelengths, qe)

Next we define the StoreScenario parameters. These parameters all have to do with the environmental factors that are present in the image and affect the image quality.

In [3]:
#sensor altitude
altitude=9000.0
#range to target
groundRange = 60000.0

scenario_name = 'niceday'
#weather model
ihaze = 1

scenario = StoreScenario(scenario_name,ihaze,altitude,groundRange)
scenario.aircraftSpeed = 100.0 #ground speed in m/s